In [1]:
%pip install torch~=2.3.0
%pip install gluonts~=0.14.4
%pip install lightning
%pip install scipy
%pip install orjson
%pip install ujson
%pip install loguru
%pip install "matplotlib~=3.8.4"
%pip install openpyxl

%pip install "pandas~=2.1.4"
%pip install "requests~=2.32.2"
%pip install "pydantic~=2.7.1"
%pip install "numpy~=1.26.4"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 7.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2
Note: you may need to restart the kernel to use updated packages.
  Using cached gluonts-0.14.4-py3-none-any.whl.metadata (9.5 kB)
Using cached gluonts-0.14.4-py3-none-any.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 832.5 kB/s eta 0:00:00 0:00:01
  Using cached lightning_utilities-0.11.2-py3-none-any.whl.metadata (4.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00a 0:00:01
Using cached lightning_utilities-0.11.2-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 51.0 MB/s eta 0:00:00
Note: you may need to restart the 

In [2]:
import pandas
from requests import EnterpriseGoalRequest, UnitRequestResource
from services import DataService, HarmonyService, AntColonyService
from models import ObjectHarmonySearch, HarmonySearch, AntColony
import torch
import os

In [4]:
def calculate_prob_transit(harmony_search: HarmonySearch, pheromoneMatrix: torch.Tensor):
    probabilities = ((1 / harmony_search.harmony_memory) ** 2) * (pheromoneMatrix ** 0.4)

    total = torch.sum(probabilities)
    prob_tensor = probabilities / total

    return prob_tensor


def generate_rho_matrix(num_row: int, num_col: int, listUnitResource: list[UnitRequestResource], isGlobal: bool):
    matrix = torch.zeros(num_row, num_col)

    for row in range(num_row):
        for col in range(num_col):
            matrix[row, col] = listUnitResource[row].employee_score

    return matrix / 100


def read_enterprise_goal_dataset():
    list_enterprise_goal = list()

    df = pandas.read_csv('import_data/enterprise_goal.csv')

    for index, row in df.iterrows():
        instance = EnterpriseGoalRequest(id=row['ID'], description=row['Description'], planed_value=row['Planed value'],
                                         unit=row['Unit'], success_criteria=row['Success criteria'])
        list_enterprise_goal.append(instance)

    return list_enterprise_goal


def read_unit_resource_dataset():
    list_unit_resource = list()

    df = pandas.read_csv('import_data/unit_resource.csv')

    for index, row in df.iterrows():
        instance = UnitRequestResource(id=index + 1, unit=row['Unit'], num_employee=row['NumEmployee'],
                                       employee_score=row['EmployeeScore'], min_score=row['minEmployeeScore'],
                                       max_score=row['maxEmployeeScore'])
        list_unit_resource.append(instance)

    return list_unit_resource


In [8]:
def main():
    global best_solution
    list_enterprise_goal = read_enterprise_goal_dataset()
    list_unit_resource = read_unit_resource_dataset()

    data_service = DataService()

    # define const
    num_row = len(list_unit_resource)
    num_col = len(list_enterprise_goal)
    num_improvisations = 5000
    hms = 100
    num_ant = 5

    rho_local = generate_rho_matrix(num_row=num_row, num_col=num_col, listUnitResource=list_unit_resource,
                                    isGlobal=False)
    rho_global = generate_rho_matrix(num_row=num_row, num_col=num_col, listUnitResource=list_unit_resource,
                                        isGlobal=False)

    # generate tensor
    lower_upper_tensor = data_service.build_lower_upper_tensor(num_row, listUnitResource=list_unit_resource)
    object_harmony_search = ObjectHarmonySearch(
        lower_upper_matrix=lower_upper_tensor, max_improvisations=num_improvisations, hms=hms)
    harmony_search = HarmonySearch(
        objective_harmony_search=object_harmony_search)

    # build harmony search solution candidate
    harmony_memory: torch.Tensor = data_service.build_hs_memory_candidate(harmony_search, lower_upper_tensor, num_row,
                                                                            num_col)

    # build ant colony model
    pheromone_matrix = torch.ones(object_harmony_search.hms, num_row, num_col)

    ant_colony = AntColony(number_ants=num_ant, pheromone_matrix=pheromone_matrix)

    # START RUN ALGORITHMS
    harmony_service = HarmonyService()
    ant_colony_service = AntColonyService(ant_colony=ant_colony)
    harmony_search.set_harmony_memory(harmony_memory=harmony_memory)

    # Assuming you have a list of generations and their corresponding fitness values
    list_solution = list()
    list_fitness = list()
    current_best_solution = None
    current_best_fitness = None
    current_best_position = None

    for _ in range(num_improvisations):
        harmony_service.run_algorithm(harmony_search, lower_upper_tensor, num_row, num_col)

        prob_transit = calculate_prob_transit(harmony_search=harmony_search, pheromoneMatrix=pheromone_matrix)

        weight, position, fitness = ant_colony_service.run_algorithm(harmony_search=harmony_search, num_row=num_row,
                                                                     num_col=num_col, prob_transit=prob_transit)

        if current_best_solution is None:
            current_best_solution = weight
            current_best_fitness = fitness
            current_best_position = position

        if current_best_solution is not None and current_best_fitness > fitness:
            current_best_solution = weight
            current_best_fitness = fitness
            current_best_position = position

        gen_solution = {
            'weight': weight,
            'position': position,
            'fitness': fitness
        }

        list_solution.append(gen_solution)
        list_fitness.append(current_best_fitness)

        ant_colony_service.update_local_pheromone(ant_colony=ant_colony, prob_transit=prob_transit, rho_local=rho_local)

        ant_colony_service.update_global_pheromone(prob_transit=prob_transit, rho_global=rho_global, positions=position,
                                                   fitness=fitness)

        print(current_best_fitness, current_best_solution.sum(dim=0))

    return torch.tensor(list_fitness), pheromone_matrix, current_best_solution, current_best_fitness


list_fitness, pheromone_matrix, best_solution, best_fitness = main()

20.1563777923584 tensor([3.6143, 3.6101, 2.3085, 3.6101, 3.4265, 3.5870])
17.979990005493164 tensor([3.6143, 3.6101, 2.3085, 2.2448, 3.4265, 2.7759])
17.979990005493164 tensor([3.6143, 3.6101, 2.3085, 2.2448, 3.4265, 2.7759])
17.979990005493164 tensor([3.6143, 3.6101, 2.3085, 2.2448, 3.4265, 2.7759])
16.18772315979004 tensor([2.7424, 2.6897, 2.3085, 2.2448, 3.4265, 2.7759])
16.18772315979004 tensor([2.7424, 2.6897, 2.3085, 2.2448, 3.4265, 2.7759])
16.18772315979004 tensor([2.7424, 2.6897, 2.3085, 2.2448, 3.4265, 2.7759])
16.18772315979004 tensor([2.7424, 2.6897, 2.3085, 2.2448, 3.4265, 2.7759])
16.18772315979004 tensor([2.7424, 2.6897, 2.3085, 2.2448, 3.4265, 2.7759])
16.18772315979004 tensor([2.7424, 2.6897, 2.3085, 2.2448, 3.4265, 2.7759])
15.633569717407227 tensor([2.7424, 2.4053, 2.3085, 2.2448, 3.1567, 2.7759])
15.633569717407227 tensor([2.7424, 2.4053, 2.3085, 2.2448, 3.1567, 2.7759])
15.633569717407227 tensor([2.7424, 2.4053, 2.3085, 2.2448, 3.1567, 2.7759])
15.633569717407227 t

In [17]:
list_enterprise_goal = read_enterprise_goal_dataset()
list_unit_resource = read_unit_resource_dataset()

list_result = list()

for row, unit_resource in enumerate(list_unit_resource):
    unit = unit_resource.unit
    for col, goal in enumerate(list_enterprise_goal):
        goal_description = goal.description
        success_criteria = goal.success_criteria
        goal_unit = goal.unit
        unit_goal_value = goal.planed_value * best_solution[row, col]

        payload = {
            'unit': unit,
            'goal_description': goal_description,
            'success_criteria': success_criteria,
            'goal_unit': goal_unit,
            'unit_goal_value': unit_goal_value.item()
        }

        list_result.append(payload)

# Tạo DataFrame từ list_result
df = pandas.DataFrame(list_result)

# Đường dẫn tới tệp CSV mà bạn muốn ghi vào
csv_file_path = "output.csv"

# Ghi DataFrame vào tệp CSV
df.to_csv(csv_file_path, index=False)